In [1]:
import numpy as np
from scipy.linalg import block_diag

In [2]:
def tsqr_4_blocks(W):
    m, n = W.shape

    print(f"Initial matrix:\n{W}")

    # Divide and factor locally
    # Split the tall matrix into 4 equal-ish blocks (W0, W1, W2, W3)
    split_indices = [m // 4, m // 2, 3 * m // 4]

    # np.split takes an ndarray and splits at split_indices
    W0, W1, W2, W3 = np.split(W, split_indices)

    # Just to verify shapes / splitting was correct
    for i, block in enumerate([W0, W1, W2, W3]):
        print(f"Block {i} shape: {block.shape}\n{block}\n")

    # Perform standard QR on each block
    Q00, R00 = np.linalg.qr(W0)
    Q10, R10 = np.linalg.qr(W1)
    Q20, R20 = np.linalg.qr(W2)
    Q30, R30 = np.linalg.qr(W3)
    # W0, W1, W2 and W3 represent the four blocks (horizontal slices) of the input
    # matrix W. Q00 and R00 represent the resulting matrices from the QR
    # decomposition of the first block of W, W0.

    # First level of the reduction tree
    # Stack the R's from Stage 1 into pairs and factor again
    Q01, R01 = np.linalg.qr(np.vstack([R00, R10]))
    Q11, R11 = np.linalg.qr(np.vstack([R20, R30]))

    # Final level of the reduction tree
    # R's from Stage 2 and do the final factorisation
    Q02, R02 = np.linalg.qr(np.vstack([R01, R11]))

    # ASSEMBLE FINAL MATRICES
    # The final R is simply the R from the very top of the tree
    R_final = R02

    # To calculate the final Q, we build the large block-diagonal
    # matrices using scipy.linalg.block_diag()
    Q_stage1 = block_diag(Q00, Q10, Q20, Q30)
    Q_stage2 = block_diag(Q01, Q11)
    Q_stage3 = Q02

    Q_final = Q_stage1 @ Q_stage2 @ Q_stage3

    return Q_final, R_final

In [3]:
# Create tall skinny matrix
np.random.seed(2003)
W_test = np.random.rand(100, 3)

Q, R = tsqr_4_blocks(W_test)

# Printing to check reduction
print(f"Original Matrix W shape: {W_test.shape}")
print(f"Final Q shape:           {Q.shape}")
print(f"Final R shape:           {R.shape}")

# Verify that Q * R equals original W
# np.allclose checks if two matrices are essentially identical
# (accounting for the tiny floating-point rounding errors).
is_correct = np.allclose(W_test, Q @ R)
print(f"\nDoes Q * R = W?          {is_correct}")

# Verify Q is orthogonal (Does Q.T * Q = Identity matrix?)
identity_check = np.allclose(np.eye(3), Q.T @ Q)
print(f"Is Q orthogonal?         {identity_check}")

Initial matrix:
[[0.49229035 0.44559396 0.40512847]
 [0.34639418 0.84708402 0.18836205]
 [0.97364912 0.83779638 0.8692328 ]
 [0.29332114 0.82832862 0.10176295]
 [0.64558901 0.44510762 0.08514436]
 [0.73660189 0.93574237 0.51640204]
 [0.84752699 0.74034111 0.62825233]
 [0.41974476 0.03846291 0.73362476]
 [0.74879572 0.45662129 0.95147925]
 [0.90752038 0.70826406 0.84235433]
 [0.21686925 0.18266415 0.21079483]
 [0.95154134 0.6657386  0.34109032]
 [0.95714893 0.2949929  0.7366212 ]
 [0.35188084 0.94431061 0.31199411]
 [0.18902452 0.74575081 0.99873952]
 [0.47176854 0.84632083 0.91073962]
 [0.64034393 0.3936972  0.93232906]
 [0.7564468  0.35072814 0.01321991]
 [0.58049615 0.82917829 0.84927372]
 [0.4912887  0.56793826 0.21309179]
 [0.25885283 0.99558613 0.40342889]
 [0.17872251 0.03521072 0.78960318]
 [0.6911912  0.62541279 0.70200676]
 [0.05032252 0.96597603 0.60880712]
 [0.86905244 0.05336506 0.1667369 ]
 [0.37750342 0.18656503 0.47693308]
 [0.77713461 0.02837799 0.13933331]
 [0.17952466